In [16]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [17]:
import numpy as np

In [18]:
columns = ["User_id", "`review/score`", "book_id"]
", ".join(columns)

'User_id, `review/score`, book_id'

In [19]:
host = 'fireteam108c-db.ctq6oauek6z5.us-east-2.rds.amazonaws.com:3306'
user = 'admin108c'
password = 'fireteam108c'
database = 'amazon_books'

In [20]:
def get_table_data(columns:list[str], table_name, uri:str, chunk_size:int=100000):
	engine = create_engine(uri)
	query = f"SELECT {", ".join(columns)} FROM {table_name}"

	with engine.connect() as connection:
		try:
			result = pd.read_sql_query(query, connection, chunksize=chunk_size)
			return pd.concat([chunk for chunk in result])
		except Exception as e:
			connection.rollback()
			connection.close()
			print("Error in reading data from database")
			print(e)

	

In [21]:
ratings_user_tagged = get_table_data(columns, "ratings_user_tagged", f"mysql+mysqldb://{user}:{password}@{host}/{database}")

In [22]:
ratings_user_tagged

,User_id,review/score,book_id
0,AVCGYZL8FQQTD,4.0,1882931173
1,A30TK6U7DNS82R,5.0,0826414346
2,A3UH4UZ4RSVO82,5.0,0826414346
3,A2MVUWT453QH61,4.0,0826414346
4,A22X4XUPKF66MR,4.0,0826414346
...,...,...,...
34968,AI1QNMVF2E3TN,5.0,B000NSLVCU
34969,AOFGOUMXLMVZS,4.0,B000NSLVCU
34970,A1SMUB9ASL5L9Y,4.0,B000NSLVCU
34971,A2AQMEKZKK5EE4,4.0,B000NSLVCU


In [23]:
res = { k: v.get(["book_id", "review/score"]) for k, v in ratings_user_tagged.groupby('User_id').filter(lambda x: len(x) > 20).groupby('User_id')}

In [24]:
reviewed_tagged_books = get_table_data(["book_id"], "reviewed_books", f"mysql+mysqldb://{user}:{password}@{host}/{database}")
reviewed_tagged_books

,book_id
0,1882931173
1,0826414346
2,0829814000
3,0595344550
4,0253338352
...,...
6706,0590482467
6707,0570047870
6708,B000OVF7JY
6709,1402508735


In [25]:
# ratings_user_tagged["book_id"].unique().shape[0]

In [26]:
# reviewed_tagged_books

In [27]:
matrix = pd.DataFrame(index=res.keys(), columns = reviewed_tagged_books["book_id"].values)
matrix

,1882931173,0826414346,0829814000,0595344550,0253338352,0802841899,B0007FIF28,B000JINSBG,0895554224,0963923080,...,B0006E1KEK,0896102130,0812553861,0971556504,1418016985,0590482467,0570047870,B000OVF7JY,1402508735,0534400604
A101446I5AWY0Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A103U0Q3IKSXHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A103W7ZPKGOCC9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A105E427BB6J65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A106016KSI0YQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZXEZRXZQL1H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZXGPM8EKSHE9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZXQKAMHK35PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZY8LGHVF8GMZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for k, v in res.items():
	t = v.reset_index(drop=True).transpose()
	matrix.loc[k][t.loc["book_id"]] = t.loc["review/score"]
# 4 min 20 sec runtime


In [29]:
matrix = matrix.fillna(0)

In [30]:
# res = { k: v.get(["book_id", "review/score"]) for k, v in ratings_user_tagged.groupby('User_id').filter(lambda x: len(x) > 10).groupby('User_id')}

In [31]:
# books_ratings = books_ratings.drop_duplicates()

In [32]:
# books_ratings_tagged = pd.read_sql_table('books_rating', con=connection)

In [33]:
matrix.T.to_csv("full_matrix.csv.gzip", compression="gzip", encoding="utf-8", chunksize=10000)

In [34]:
matrix

,1882931173,0826414346,0829814000,0595344550,0253338352,0802841899,B0007FIF28,B000JINSBG,0895554224,0963923080,...,B0006E1KEK,0896102130,0812553861,0971556504,1418016985,0590482467,0570047870,B000OVF7JY,1402508735,0534400604
A101446I5AWY0Z,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
A103U0Q3IKSXHE,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
A103W7ZPKGOCC9,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
A105E427BB6J65,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
A106016KSI0YQ,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZXEZRXZQL1H2,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
AZXGPM8EKSHE9,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
AZXQKAMHK35PA,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0
AZY8LGHVF8GMZ,0,0.0,0.0,0,0,0,0,0,0,0.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0


In [38]:
test = pd.read_csv("matrix.csv.gzip", compression='gzip', index_col=0)
test

,A01038432MVI9JXYTTK5T,A07587253PDB1WFIEPBPY,A100NGGXRQF0AQ,A100V1W0C8BWOL,A100YDQ9DIP06P,A100YHBWL4TR4D,A1012N48J0Z65N,A101446I5AWY0Z,A1016MYYF5QSTY,A101BVV4DR3G81,...,AZXFUIA3GYE0E,AZXGPM8EKSHE9,AZXQKAMHK35PA,AZY0QGEN37C9L,AZY8LGHVF8GMZ,AZY96LU53VWGO,AZYIHKBD3FZBI,AZZ1E6SG4RAT7,AZZCSTUZ5EO4I,AZZVZL4QEHEHO
0826414346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0829814000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0963923080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0854968350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0918973031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0006E1KEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0812553861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418016985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0590482467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# matrix.replace(0, np.nan).isna().sum().sum()
test.notna().sum().sum()

548316